# Correct Defined Space Coordinates

In [1]:
# these are the labels I have given the lines in my pic (A=0, B=1,... etc)
lines = [0,1,2,3,4,5]

# every space on the board can be uniquely described by the 2 lines that intersect at that space
# however, some line intersections do not exist since the shape is a star, so I manually type them here
spaces_list = [(0,1),(0,2),(0,3),(0,4),
               (1,2),(1,5),(1,4),
               (2,3),(2,5),
               (3,4),(3,5),
               (4,5)]
spaces_list.sort()  #order the spaces
print(spaces_list)

# create a dict to track which numbers we have placed in each space (starting them as 0 for default)
my_gameboard = {space:0 for space in spaces_list}

# make a dictionary that tells us the spaces contained in each line
line_spaces = {l:[space for space in my_gameboard.keys() if l in space] for l in lines}
print(line_spaces)



[(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 4), (1, 5), (2, 3), (2, 5), (3, 4), (3, 5), (4, 5)]
{0: [(0, 1), (0, 2), (0, 3), (0, 4)], 1: [(0, 1), (1, 2), (1, 4), (1, 5)], 2: [(0, 2), (1, 2), (2, 3), (2, 5)], 3: [(0, 3), (2, 3), (3, 4), (3, 5)], 4: [(0, 4), (1, 4), (3, 4), (4, 5)], 5: [(1, 5), (2, 5), (3, 5), (4, 5)]}


# Object Oriented

In [2]:
# function converting integers to 2 digit strings
def myconv(x):
    if x < 10:
        return '0' + str(x)
    else:
        return str(x)


In [3]:
class Shrey:
    def __init__(self):
        # these are the labels I have given the lines in my pic (A=0, B=1,... etc)
        self.lines = [0,1,2,3,4,5]
        # every space on the board can be uniquely described by the 2 lines that intersect at that space
        # however, some line intersections do not exist since the shape is a star, so I manually type them here
        self.spaces_list = [(0,1),(0,2),(0,3),(0,4),
                    (1,2),(1,5),(1,4),
                    (2,3),(2,5),
                    (3,4),(3,5),
                    (4,5)]
        self.spaces_list.sort()  #order the spaces
        # create a dict to track which numbers we have placed in each space (starting them as 0 for default)
        self.gameboard = {space:0 for space in self.spaces_list}
        # make a dictionary that tells us the spaces contained in each line
        self.line_spaces = {l:[space for space in self.gameboard.keys() if l in space] for l in self.lines}
        # numbers used to solve
        nums = [1,2,3,4,5,6,7,8,9,10,11,12]
        #start avail_nums as listing all numbers available for all spaces
        self.avail_nums = {space:nums for space in self.gameboard}
        # create a history of which numbers we have placed in which spaces so far
        # the history is a list of tuples (space,n), telling us which n has been placed in each space
        # the 0th element will be the first placement we did, and ith element is the ith
        self.history = []
        # space the backtracking algo is currently trying to fill
        self.focus_space = (0,1)  #initialise as 0,1

    # method to view the star puzzle, with visualisation of what each line sums to
    def view(self):
        line_sums = [sum([self.gameboard[space] for space in self.line_spaces[l]]) for l in self.line_spaces]    
        print(f'''
                       (={line_sums[0]})
           (={line_sums[3]})       {myconv(self.gameboard[(0,1)])}  
              {myconv(self.gameboard[(2,3)])}   {myconv(self.gameboard[(0,2)])}   {myconv(self.gameboard[(1,2)])}   {myconv(self.gameboard[(2,5)])} (={line_sums[2]}) 
                 {myconv(self.gameboard[(0,3)])}        {myconv(self.gameboard[(1,5)])}
        (={line_sums[4]})  {myconv(self.gameboard[(0,4)])}   {myconv(self.gameboard[(3,4)])}   {myconv(self.gameboard[(4,5)])}   {myconv(self.gameboard[(1,4)])} 
                      {myconv(self.gameboard[(3,5)])}      (={line_sums[1]})
                  (={line_sums[5]})
            
        '''
        )
    
    # function to check if the board is solved or not
    def board_solved(self):
        # sum the lines
        line_sums = [sum([self.gameboard[space] for space in self.line_spaces[l]]) for l in self.line_spaces]    
        # if they are all 26 then the board is solved
        if line_sums == [26 for l in self.line_spaces]:
            return True
        else:
            return False

    # true or false, can we pace this number in this space?
    def canplace_noloop(self,space,n):
        # copy gameboard
        gameboard_copy = self.gameboard.copy()
        # place it and see if it cause the lines to sum to more than 26
        gameboard_copy[space] = n
        # sum the lines this space is part of
        sums = [sum([gameboard_copy[spacei] for spacei in self.line_spaces[l]]) for l in space]
        if True in [x>26 for x in sums]:
            return False
        # if both lines sum to less than 26, return true
        return True

    # grab the latest free space in avail_nums and loop though putting the available numbers in it
    def tryplace_noloop(self,n_last=None, space_last=None):
        # grab the first space that has 0 as its number (i.e. the first blank space)
        space = [space for space in self.gameboard if self.gameboard[space] == 0][0]
        # loop through the available numbers for this space and try placing them
        for n in self.avail_nums[space]:
            if self.canplace_noloop(space, n):
                # place n in the free space since we know we can!
                self.gameboard[space] = n  
                # last n is this n
                n_last = n
                # last space is this space
                space_last = space
                # add this placement to history
                self.history += [(space_last,n_last)]
                # remove n from the available numbers of any remaining free spaces
                self.avail_nums.update(
                    {empty_space:[x for x in self.avail_nums[empty_space] if  x!= n] for empty_space in [
                        space for space in self.gameboard if (self.gameboard[space] == 0) and space != space_last]
                        }
                )
                return
        # only reach here if none of the available numbers can go in this space, so we need to backtrack
        self.gameboard[space_last] = 0  #reset last space, since the last space made the puzzle unsolveable
        self.avail_nums[space_last] = [x for x in self.avail_nums[space_last] if x != n_last]  #remove n_last from avail_nums for this space
        # now pull the numbers that haven't been used yet
        used = [x for x in self.gameboard.values() if x!= 0]
        not_used = list(set([1,2,3,4,5,6,7,8,9,10,11,12]) - set(used))  #utilising set-minus
        # now reset the avail_nums of all spaces ahead of the last space, so that their avail_nums is now the not_used list
        self.avail_nums.update({empty_space: not_used for empty_space in [
            space for space in self.gameboard if (self.gameboard[space] == 0) and space != space_last]
            }
        )
        # now grab n_last and space_last, which correspond to the latest space that has been filled
        filled_spaces = [space for space in self.gameboard if self.gameboard[space] != 0]
        filled_spaces.sort()  #sort it so our algorithm does each space in increasing order
        # grab the first one if the list isn't empty
        if filled_spaces != []:
            space_last = filled_spaces[-1]
            n_last = self.gameboard[space_last]
            return
        else:
            # else we have the case where space_last is (0,0) i.e. the first space in the puzzle, so reset n_last and space_last to None
            n_last = None
            space_last = None
    
    # tryplace but using self.history
    def tryplace_whist(self):
        # grab the first space that has 0 as its number (i.e. the first blank space)
        space = [space for space in self.gameboard if self.gameboard[space] == 0][0]
        # loop through the available numbers for this space and try placing them
        for n in self.avail_nums[space]:
            if self.canplace_noloop(space, n):
                # place n in the free space since we know we can!
                self.gameboard[space] = n  
                # add this placement to history
                self.history += [(space,n)]
                # remove n from the available numbers of any remaining free spaces
                self.avail_nums.update(
                    {empty_space:[x for x in self.avail_nums[empty_space] if  x!= n] for empty_space in [
                        spacei for spacei in self.gameboard if (self.gameboard[spacei] == 0) and spacei != space]
                        }
                )
                return
        # only reach here if none of the available numbers can go in this space, so we need to backtrack
        # grab most recent placement from history, or if there is none then return
        (space_last,n_last) = self.history[-1]
        self.gameboard[space_last] = 0  #reset last space, since the last space made the puzzle unsolveable
        self.avail_nums[space_last] = [x for x in self.avail_nums[space_last] if x != n_last]  #remove n_last from avail_nums for this space
        # and now remove this from history
        self.history = self.history[:-1]
        # now pull the numbers that haven't been used yet
        used = [x for x in self.gameboard.values() if x!= 0]
        not_used = list(set([1,2,3,4,5,6,7,8,9,10,11,12]) - set(used))  #utilising set-minus
        # now reset the avail_nums of all spaces ahead of the last space, so that their avail_nums is now the not_used list
        self.avail_nums.update({empty_space: not_used for empty_space in [
            spacei for spacei in self.gameboard if (self.gameboard[spacei] == 0) and spacei != space_last]
            }
        )

    # tryplace but using self.history and focus_space
    def tryplace_focus(self):
        # grab the focus_space
        space = self.focus_space
        # loop through the available numbers for this space and try placing them
        for n in self.avail_nums[space]:
            if self.canplace_noloop(space, n):
                # place n in the free space since we know we can!
                self.gameboard[space] = n  
                # add this placement to history
                self.history += [(space,n)]
                # remove n from the available numbers of any remaining free spaces
                self.avail_nums.update(
                    {empty_space:[x for x in self.avail_nums[empty_space] if  x!= n] for empty_space in [
                        spacei for spacei in self.gameboard if (self.gameboard[spacei] == 0) and spacei != space]
                        }
                )
                # and set the focus space to be the next free space from the sorted list of free spaces
                free_spaces = [spacei for spacei in self.gameboard if (self.gameboard[spacei] == 0)]
                if free_spaces == []:
                    return  #leave function if no free spaces in puzzle
                free_spaces.sort()
                self.focus_space = free_spaces[0]
                return
        # only reach here if none of the available numbers can go in this space, so we need to backtrack
        # grab most recent placement from history, or if there is none then return
        (space_last,n_last) = self.history[-1]
        # the last space now becomes the focus space, since we are trying to fill that one again
        self.focus_space = space_last
        self.gameboard[space_last] = 0  #reset last space, since the last space made the puzzle unsolveable
        self.avail_nums[space_last] = [x for x in self.avail_nums[space_last] if x != n_last]  #remove n_last from avail_nums for this space
        # and now remove this from history
        self.history = self.history[:-1]
        # now pull the numbers that haven't been used yet
        used = [x for x in self.gameboard.values() if x!= 0]
        not_used = list(set([1,2,3,4,5,6,7,8,9,10,11,12]) - set(used))  #utilising set-minus
        # now reset the avail_nums of all spaces ahead of the last space, so that their avail_nums is now the not_used list
        self.avail_nums.update({empty_space: not_used for empty_space in [
            spacei for spacei in self.gameboard if (self.gameboard[spacei] == 0) and spacei != space_last]
            }
        )

    # finally define a function which will iteratively place numbers until it arrives at a valid solution
    def solve_noloop(self):

        # now enter a while, which we will only leave if we solve the puzzle
        count = 0
        while (self.board_solved() == False):
            self.tryplace_focus()
            count+=1
        
        # only reach here if we solve it
        print(f"solved in {count} iterations")
            






In [7]:
b = Shrey()

In [10]:
b.gameboard

{(0, 1): 0,
 (0, 2): 0,
 (0, 3): 0,
 (0, 4): 0,
 (1, 2): 0,
 (1, 4): 0,
 (1, 5): 0,
 (2, 3): 0,
 (2, 5): 0,
 (3, 4): 0,
 (3, 5): 0,
 (4, 5): 0}

In [12]:
b.avail_nums

{(0, 1): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 (0, 2): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 (0, 3): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 (0, 4): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 (1, 2): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 (1, 4): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 (1, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 (2, 3): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 (2, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 (3, 4): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 (3, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 (4, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}

In [4]:
a = Shrey()
a.view()



                       (=0)
           (=0)       00  
              00   00   00   00 (=0) 
                 00        00
        (=0)  00   00   00   00 
                      00      (=0)
                  (=0)
            
        


In [5]:
a.solve_noloop()


solved in 1914554 iterations


In [6]:
a.view()


                       (=26)
           (=26)       01  
              08   02   09   07 (=26) 
                 11        10
        (=26)  12   03   05   06 
                      04      (=26)
                  (=26)
            
        


In [11]:
a.gameboard

{(0, 1): 1,
 (0, 2): 2,
 (0, 3): 11,
 (0, 4): 12,
 (1, 2): 9,
 (1, 4): 6,
 (1, 5): 10,
 (2, 3): 8,
 (2, 5): 7,
 (3, 4): 3,
 (3, 5): 4,
 (4, 5): 5}

## rough

In [56]:
# # sum the lines
# line_sums = [sum([gameboard[space] for space in line_spaces[l]]) for l in line_spaces]    


In [106]:
# # (0,1) at top of star,
# # next row is (2,3) then (0,2) then (1,2) then (2,5)
# # next row is (0,3) then (1,5)
# # next row is (0,4) then (3,4) then (4,5) then (1,4)
# # finally (3,5)

# viewstring = f'''
#             (={line_sums[0]})
#    (={line_sums[3]})     {a.gameboard[(0,1)]}  
#       {a.gameboard[(2,3)]}   {a.gameboard[(0,2)]}   {a.gameboard[(1,2)]}   {a.gameboard[(2,5)]} (={line_sums[2]}) 
#         {a.gameboard[(0,3)]}       {a.gameboard[(1,5)]}
# (={line_sums[4]})  {a.gameboard[(0,4)]}   {a.gameboard[(3,4)]}   {a.gameboard[(4,5)]}   {a.gameboard[(1,4)]} 
#             {a.gameboard[(0,1)]}      (={line_sums[0]})
      
# '''

# print(viewstring)



            (=0)
   (=0)     0  
      0   0   0   0 (=0) 
        0       0
(=0)  0   0   0   0 
            0      (=0)
      

